In [25]:
import networkx as nx
import matplotlib.pyplot as plt

G= nx.read_gml("./lesmis.gml") # Read the network
G = G.to_undirected() #Remove a direcao dos links


In [32]:
attrs = {}
N = len(G) #number of nodes in Graph

for n in range(0, N):
    if (n%2): attrs[n] = {"P": 0, "Behaviour": "Cooperator"}
    else:  attrs[n] = {"P": 0, "Behaviour": "Detractor"}

print(attrs)

nx.set_node_attributes(G, attrs)
nx.write_gml(G, "./lesmis2.gml")

{0: {'P': 0, 'Behaviour': 'Detractor'}, 1: {'P': 0, 'Behaviour': 'Cooperator'}, 2: {'P': 0, 'Behaviour': 'Detractor'}, 3: {'P': 0, 'Behaviour': 'Cooperator'}, 4: {'P': 0, 'Behaviour': 'Detractor'}, 5: {'P': 0, 'Behaviour': 'Cooperator'}, 6: {'P': 0, 'Behaviour': 'Detractor'}, 7: {'P': 0, 'Behaviour': 'Cooperator'}, 8: {'P': 0, 'Behaviour': 'Detractor'}, 9: {'P': 0, 'Behaviour': 'Cooperator'}, 10: {'P': 0, 'Behaviour': 'Detractor'}, 11: {'P': 0, 'Behaviour': 'Cooperator'}, 12: {'P': 0, 'Behaviour': 'Detractor'}, 13: {'P': 0, 'Behaviour': 'Cooperator'}, 14: {'P': 0, 'Behaviour': 'Detractor'}, 15: {'P': 0, 'Behaviour': 'Cooperator'}, 16: {'P': 0, 'Behaviour': 'Detractor'}, 17: {'P': 0, 'Behaviour': 'Cooperator'}, 18: {'P': 0, 'Behaviour': 'Detractor'}, 19: {'P': 0, 'Behaviour': 'Cooperator'}, 20: {'P': 0, 'Behaviour': 'Detractor'}, 21: {'P': 0, 'Behaviour': 'Cooperator'}, 22: {'P': 0, 'Behaviour': 'Detractor'}, 23: {'P': 0, 'Behaviour': 'Cooperator'}, 24: {'P': 0, 'Behaviour': 'Detractor'